In [6]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spinless_fermion_basis_1d
from pyscf import gto, scf
import numpy as np
from scipy.linalg import eigh, cholesky

def spinorb_from_spatial(one_body_integrals, o_integral, two_body_integrals, EQ_TOLERANCE = 1e-15):
    n_qubits = 2 * one_body_integrals.shape[0]
    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))
    S_matrix = np.zeros((n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):
            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[p, q]
            one_body_coefficients[2 * p + 1, 2 * q + 1] = one_body_integrals[p, q]
            S_matrix[2 * p, 2 * q] = o_integral[p, q]
            S_matrix[2 * p + 1, 2 * q + 1] = o_integral[p, q]
            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):
                    # Mixed spin
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r + 1 , 2 * s] = two_body_integrals[
                        p, q, r, s
                    ]
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r, 2 * s + 1] = two_body_integrals[
                        p, q, r, s
                    ]

                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 * s] = two_body_integrals[
                        p, q, r, s
                    ]
                    two_body_coefficients[
                        2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1
                    ] = two_body_integrals[p, q, r, s]

    # Truncate.
    one_body_coefficients[np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.0
    two_body_coefficients[np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.0
    S_matrix[np.absolute(S_matrix) < EQ_TOLERANCE] = 0.0
    return one_body_coefficients,S_matrix, two_body_coefficients



# Define the molecule
mol = gto.M(
    atom='''
O  0.0000000  0.0000000  0.0000000
H  0.0000000  0.7570000  0.5860000
H  0.0000000 -0.7570000  0.5860000
''',
    basis='sto3g', 
    #spin = 1,
    unit='angstrom' 
)

# Perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()
print(f'N_ao = {mol.nao}')
print(f'N_ele = {mol.tot_electrons()}')
n_qubits = int(2 * mol.nao)
n_ele = int(mol.tot_electrons())
e_nuc = mol.energy_nuc()

converged SCF energy = -74.9629466565387
N_ao = 7
N_ele = 10


For HF method, we have:
$$
 (\ket{\psi_1},\cdots,\ket{\psi_{2L}}) = (\ket{\phi_1},\cdots,\ket{\phi_{2L}}) \Phi,
$$
The Fock operator:
$$
\hat{f}(r_i) = \hat{h}(r_i)+ \hat{V}(r_i)-\hat{K}(r_i),
$$
and calculation of the Fock matrix elements:
$$
F_{kl} = h_{kl}+V_{kl}-K_{kl},
$$
where the single electron integral term reads
$$ 
h_{kl} = \langle \phi_k|\hat{h}|\phi_l\rangle = (i|h|j).
$$
For the Fock potential terms, they depend on the molecular orbital coefficients $\Phi$. Formally, we have:
$$
\hat{V}[\Phi] \ket{\phi_l} = \sum_{j=1}^{2L}\langle \psi_j |r^{-1} |\psi_j\rangle \ket{\phi_l},
$$
$$
\hat{K}[\Phi]\ket{\phi_l} = \sum_{j=1}^{2L}\langle \psi_j | r^{-1}|\phi_l\rangle \ket{\psi_j}.
$$ 

Therefore we can calculate the matrix elements
$$
\begin{align*}
V_{kl}  &= \langle \phi_k |\hat{V}|\phi_l\rangle \\&=\sum_{pq} (\widetilde{\Phi}\widetilde \Phi^\dagger)_{p,q} \int \phi_k(1)\phi_p(2)r_{12}^{-1}\phi_l(1)\phi_q(2)\mathrm{d}1\mathrm{d}2\\
&=\sum_{pq} (\widetilde{\Phi}\widetilde \Phi^\dagger)_{p,q} r_{kplq}
\end{align*}
$$
$$
\begin{align*}
K_{kl}  &= \langle \phi_k |\hat{V}|\phi_l\rangle \\&=\sum_{pq} (\widetilde{\Phi}\widetilde \Phi^\dagger)_{p,q} \int \phi_k(1)\phi_p(2)r_{12}^{-1}\phi_q(1)\phi_l(2)\mathrm{d}1\mathrm{d}2\\
&=\sum_{pq} (\widetilde{\Phi}\widetilde \Phi^\dagger)_{p,q} r_{kpql}
\end{align*}
$$
Note that the coefficient matrix (density matrix) only considers the part related to the first $N$ particles.

In [7]:
##### define molecular Hamiltonians ###


# read 1- and 2- electron integrals and overlap integrals
e1, ovlp, eri = mol.intor('int1e_nuc')+mol.intor('int1e_kin'), mol.intor('int1e_ovlp'), mol.intor('int2e')
# need to be checked later!!!
# read: (ij|kl) -> (ik|jl) = <ij|kl> (if the spin is right)
eri = np.transpose(eri ,(0,2,1,3)) 


e1_spin, ovlp_spin, eri_spin = spinorb_from_spatial(e1, ovlp, eri)


## initialize coefficient matrix \Phi and density matrix \rho
Phi = np.zeros((n_qubits,n_qubits))
rho = np.dot(Phi[:,:n_ele],Phi[:,:n_ele].T)

## compute the fock operators
# fock_kl = h_kl + V_kl - K_kl
def get_fock(rho, e1_spin, eri_spin):
    fock = e1_spin + np.einsum('pq, kplq', rho, eri_spin, optimize = True) - np.einsum('pq, kpql', rho, eri_spin, optimize = True)
    return(fock)

def get_energy(rho, e1_spin, eri_spin):
    return np.einsum('kl,kl',rho,e1_spin)\
             +0.5*np.einsum('kl,pq,kplq',rho,rho,eri_spin,optimize=True)\
             -0.5*np.einsum('kq,pl,kplq',rho,rho,eri_spin,optimize=True)

def get_first_order(rho, eri_spin):
    return -0.5*np.einsum('kl,pq,kplq',rho,rho,eri_spin,optimize=True)\
             +0.5*np.einsum('kq,pl,kplq',rho,rho,eri_spin,optimize=True)

SCF iteration



Let $\ket{\Psi}$ be the Slater determinant constructed using $(\ket{\psi_1},\cdots,\ket{\psi_N})$, then the HF energy will be (note that for any single Slater determinant, the SC rules also hold)
$$
E_{\mathrm{HF}} = \langle \Psi | \hat H| \Psi\rangle = \sum_a \langle\psi_a |h |\psi_a \rangle +\frac{1}{2}\sum_{ab}[\psi_a\psi_a|\psi_b\psi_b]-[\psi_a\psi_b|\psi_b\psi_a].
$$
Here, $[\cdot\cdot|\cdot\cdot]$ denotes the chemists' notation of double-electron integrals. We expand this expression into terms of electron integrals of atomic (basis) orbitals:
$$
E_{\mathrm{HF}} = \sum_k\sum_l(\widetilde\Phi\widetilde\Phi^\dagger)_{kl}h_{kl}+\frac{1}{2}\sum_{kplq}(\widetilde\Phi\widetilde\Phi^\dagger)_{kl}(\widetilde\Phi\widetilde\Phi^\dagger)_{pq}r_{kplq}-(\widetilde\Phi\widetilde\Phi^\dagger)_{kq}(\widetilde\Phi\widetilde\Phi^\dagger)_{pl}r_{kplq}
$$

In [8]:
## SCF iterations

last_energy = 0
cycle = 0
while True:
    cycle += 1
    fock = get_fock(rho, e1_spin, eri_spin)
    # solve FC = SCE, renew Phi
    e_val, Phi = eigh(fock,ovlp_spin)
    # calculate new density matrix
    rho = np.dot(Phi[:,:n_ele],Phi[:,:n_ele].T)
    # calculate energy
    energy = get_energy(rho, e1_spin, eri_spin)
    print('{} iteration, ground energy = {:5f}'.format(cycle,energy+e_nuc))
    if np.abs(energy-last_energy)<1e-15:
        print('Energy Converged,')
        print('The HF energy is {:5f}'.format(energy+e_nuc))
        break
    else:
        last_energy = energy
print(energy+e_nuc)

1 iteration, ground energy = -73.232527
2 iteration, ground energy = -74.945877
3 iteration, ground energy = -74.962097
4 iteration, ground energy = -74.962831
5 iteration, ground energy = -74.962927
6 iteration, ground energy = -74.962943
7 iteration, ground energy = -74.962946
8 iteration, ground energy = -74.962947
9 iteration, ground energy = -74.962947
10 iteration, ground energy = -74.962947
11 iteration, ground energy = -74.962947
12 iteration, ground energy = -74.962947
13 iteration, ground energy = -74.962947
14 iteration, ground energy = -74.962947
15 iteration, ground energy = -74.962947
16 iteration, ground energy = -74.962947
17 iteration, ground energy = -74.962947
18 iteration, ground energy = -74.962947
19 iteration, ground energy = -74.962947
20 iteration, ground energy = -74.962947
21 iteration, ground energy = -74.962947
22 iteration, ground energy = -74.962947
23 iteration, ground energy = -74.962947
Energy Converged,
The HF energy is -74.962947
-74.96294665654037


### Constructing the Converged HF Hamiltonian

To construct the converged Hartree-Fock (HF) Hamiltonian, we start by noting that the HF state is the ground state of the following Hamiltonian:
$$
\hat{H}_0 = \sum_{i=1}^N \hat{f}(r_i).
$$
Here, $\hat{f}(r_i)$ is the Fock operator for the $i$-th particle.

We now move to the second quantization framework, which avoids dealing directly with Slater determinants by using fermionic operators. In second quantization, we assume a set of orthonormal basis (spin orbitals) $\{\varphi_i\}_{i=1}^{2L}$. Using this basis, we construct the Fock space $\mathcal{F}$, or in our case, a sector of $\mathcal{F}$ with a specific particle number $\mathcal{F}_N$. In this setting, $\hat{H}_0$ becomes:
$$
\hat{H}_0 = \sum_{p,q=1}^{2L} F_{pq} a_p^\dagger a_q,
$$
where $F_{pq}$ are the matrix elements of the Fock operator within the orthonormal basis.

As we know, molecular orbitals $\{\psi_i\}_{i=1}^{2L}$ are commonly used orthonormal bases in quantum chemistry. We denote the fermionic operators as $\{c_p^\dagger, c_p\}_p$. Since the HF state is precisely the single Slater determinant constructed using the molecular orbitals, in second quantization, it is given by:
$$
\ket{\mathrm{HF}} = \prod_{i=1}^N c_i^\dagger \ket{\mathrm{vac}} = \ket{\underbrace{1\cdots 1}_{N}\underbrace{0\cdots0}_{2L-N}}.
$$
The Hamiltonian $\hat{H}_0$ in this basis takes the form:
$$
\hat{H}_0 = \sum_{k=1}^{2L} \varepsilon_k c_k^\dagger c_k,
$$
where $\varepsilon_k$ represents the $k$-th molecular orbital energy. Additionally, since $\ket{\mathrm{HF}} = \ket{\underbrace{1\cdots 1}_{N}\underbrace{0\cdots0}_{2L-N}}$, the Hamiltonian matrix $H$ in the fermionic basis of the $N$-particle sector gives the HF energy as $E_{\mathrm{HF}} = \langle \mathrm{HF}|\hat{H}|\mathrm{HF}\rangle = H_{00}$.

We can also directly construct the HF Hamiltonian using the converged Fock operator from the self-consistent field (SCF) process. However, we need to orthogonalize the basis set to facilitate the second-quantization representation. Given that the overlap matrix is Hermitian and non-singular, we can decompose it as:
$$
S = U s U^\dagger,
$$
where $U$ is a unitary matrix, and $s$ is a diagonal matrix of eigenvalues. We define the transformation matrix $X = U s^{-1/2}$ and transform the Fock operator accordingly:
$$
F \mapsto X^\dagger F X.
$$
This transformation is equivalent to using the orthogonalized (atomic) orbitals as the basis set. Thus, the HF state is the ground state of the quadratic Hamiltonian:
$$
\hat{H}_0 = \sum_{p,q=1}^{2L} F_{pq} a_p^\dagger a_q,
$$
(within the orthogonalized basis set), and the ground state energy is $\sum_{k=1}^N \varepsilon_k$.

### Remarks

1. The ground state of any quadratic Hamiltonian (or non-interacting Hamiltonian) $\sum_{p,q=1}^{2L} F_{pq} a_p^\dagger a_q$ is a single determinant. Diagonalizing $(F)_{pq}$ yields a unitary matrix $\Phi$ and eigenvalues $\varepsilon_k$. Implementing a basis rotation to the fermionic operators:
   $$
   c_k^\dagger = (a_1^\dagger, \cdots, a_{2L}^\dagger) \Phi, \quad c_k = (a_1, \cdots, a_{2L}) \overline{\Phi},
   $$
   transforms the Hamiltonian to:
   $$
   \hat{H}_0 = \sum_k \varepsilon_k c_k^\dagger c_k,
   $$
   and the ground state becomes $\prod_i^N c_i^\dagger \ket{\mathrm{vac}}$.

2. Generally, $\sum_{i=1}^N \varepsilon_k \neq E_{\mathrm{HF}}$. However, the correct HF energy can be retrieved by adding the first-order energy $E_1$ (defined previously in the program):
   $$
   E_{\mathrm{HF}} = \sum_{i=1}^N \varepsilon_k + E_1.
   $$


In [9]:
s, U = np.linalg.eigh(ovlp_spin)
sqrt_sinv = np.diag(np.sqrt(np.reciprocal(s)))
X = U @ sqrt_sinv
fock_pr = X.T @ fock @ X
e_orb = eigh(fock_pr)[0]
print(e_orb)


sum_ei = 0
occ = 0
for i in range(len(e_orb)):
    if e_orb[i] > 0:
        print(f'occ = {occ}')
        print(f'sum_ei = {sum_ei}')
        break
    else:
        occ += 1
        sum_ei += e_orb[i]

sum_ei - 0.5*np.einsum('kl,pq,kplq',rho,rho,eri_spin,optimize=True)\
             +0.5*np.einsum('kq,pl,kplq',rho,rho,eri_spin,optimize=True)+e_nuc


## quspin
fermbas_mol = spinless_fermion_basis_1d(n_qubits, Nf = n_ele) # restrict to the sector
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

## test
Hfock = []
for i in range(n_qubits):
    for j in range(n_qubits):
        if abs(fock_pr[i][j]) > 1e-15:
            Hfock.append([fock_pr[i][j], i, j]) # a_i^+ a_j, '+-'
fermbas_test = spinless_fermion_basis_1d(n_qubits, Nf = n_ele) # restrict to the 2 sector
H_fock = hamiltonian([['+-', Hfock]],[], basis=fermbas_test, dtype=np.float64,**no_checks)
H_mat = np.array(H_fock.todense())
print(H_mat.shape[0])
e, evc = np.linalg.eigh(H_mat)
print(e[0])
HFGS = evc.T[0]

HF_energy = e[0]  + get_first_order(rho, eri_spin) + e_nuc
print(HF_energy)


[-20.24176268 -20.24176268  -1.26836089  -1.26836089  -0.61786318
  -0.61786318  -0.4529991   -0.4529991   -0.39124291  -0.39124291
   0.60557624   0.60557624   0.74224475   0.74224475]
occ = 10
sum_ei = -45.94445752567558
1001
-45.94445752567558
-74.96294665735195


A toy model
$$
\hat{H}_0 = -3 a_0^\dagger a_0 - 2a_1^\dagger a_1-\cdots + 3a_5^\dagger a_5.
$$

In [10]:
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
single_e = [[-3, 0, 0], [-2, 1, 1], [-1, 2, 2], [1, 3, 3], [2, 4, 4], [3, 5, 5]]
fermbas_test = spinless_fermion_basis_1d(6, Nf = 5) # restrict to the 2 sector
H_op = hamiltonian([['+-', single_e]],[], basis=fermbas_test, dtype=np.float64,**no_checks)
H_mat = np.array(H_op.todense())
print(H_mat.shape[0])
e, evc = np.linalg.eigh(H_mat)
print(fermbas_test)
print(e[0])
print(H_mat[0][0])
print(evc.T[0])

6
reference states: 
array index   /   Fock state   /   integer repr. 
     0.         |1 1 1 1 1 0>           62  
     1.         |1 1 1 1 0 1>           61  
     2.         |1 1 1 0 1 1>           59  
     3.         |1 1 0 1 1 1>           55  
     4.         |1 0 1 1 1 1>           47  
     5.         |0 1 1 1 1 1>           31  
-3.0
-3.0
[1. 0. 0. 0. 0. 0.]
